In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget  
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [3]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [4]:
dfAdmDiff = dfAdm.diff().iloc[1:]
plotDates = curDates[1:]

In [5]:
dfCase = pd.DataFrame(columns=dfAdm.columns)
dfTest = pd.DataFrame(columns=dfAdm.columns)
dfPosP = pd.DataFrame(columns=dfAdm.columns)

In [6]:
# Collect the cases, tests and positive percentage from all the "cases by age" files

ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])
        curdf = pd.read_csv(latestdir+'/Cases_by_age.csv',delimiter = ';',dtype=str)
        
        curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
        curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))
        
        
        dfCase.loc[latestDate] = curdf['Antal_bekræftede_COVID-19'].values
        dfTest.loc[latestDate] = curdf['Antal_testede'].values
        dfPosP.loc[latestDate] = curdf['Procent_positive'].values

allDates = dfCase.index


In [7]:
# dfCaseDiff.tail()
# dfCase

In [8]:
dfCaseDiff = dfCase.diff().iloc[1:]
dfTestDiff = dfTest.diff().iloc[1:]
dfPosPDiff = dfPosP.diff().iloc[1:]
plotDatesCase = allDates[1:]

In [9]:
fig,ax1 = plt.subplots(1,1)

howManyToUse = len(dfCaseDiff.columns)-4
cmap = plt.cm.get_cmap('Dark2',howManyToUse)
for k in range(0,howManyToUse):
    curColName = dfCaseDiff.columns[k]
    ax1.plot(plotDatesCase,dfCaseDiff[curColName],'.-',linewidth=0.4,label=curColName,color=cmap(k))
    # meanWidth = 7
    # ax1.plot(rnTime(plotDatesCase,meanWidth),rnMean(dfCaseDiff[curColName],meanWidth),label=curColName,color=cmap(k))


# testMonday = np.datetime64('2021-05-17')
testMonday = np.datetime64('2021-03-01')
numMondays = 30
for m in range(0,numMondays):
    ax1.bar(testMonday+np.timedelta64(m*7,'D'),1000,color='grey')

ax1.legend()

ax1.set_ylim(bottom=0,top=1000)
# ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-05-05')])
ax1.set_xlim(left=np.datetime64('2021-03-01'))
ax1.set_xlim(right=np.datetime64(date.today()))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# Compare years
allDatesDiff = allDates[1:]

fig,ax1 = plt.subplots(1,1)
prevYearIndex = (allDatesDiff < np.datetime64('2021-01-01'))
prevYearDates = allDatesDiff[prevYearIndex]
prevYearSum = dfCaseDiff[prevYearIndex]['I alt']

thisYearIndex = (allDatesDiff >= np.datetime64('2021-01-01'))
thisYearDates = allDatesDiff[thisYearIndex]
thisYearSum = dfCaseDiff[thisYearIndex]['I alt']

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum)
ax1.plot(thisYearDates,thisYearSum)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:

curdf = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
# curdf

In [12]:
curdf = curdf.iloc[:-2]
# curdf['NewPositive'] = pd.to_numeric(curdf['NewPositive'])
curdf['NewPositive'] = pd.to_numeric(curdf['NewPositive'].astype(str).apply(lambda x: x.replace('.','')))
curdf['Tested'] = pd.to_numeric(curdf['Tested'].astype(str).apply(lambda x: x.replace('.','')))
curdf['PosPct'] = pd.to_numeric(curdf['PosPct'].astype(str).apply(lambda x: x.replace(',','.')))

testDates =  pd.to_datetime(curdf.Date,format='%Y-%m-%d')
# curdf

In [13]:
# plt.figure()
# curdf

In [14]:

fig,ax1 = plt.subplots(1,1)
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = curdf[prevYearIndex]['NewPositive']

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = curdf[thisYearIndex]['NewPositive']

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,label='2020')
ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],label='2021')
# ax1.plot(thisYearDates,thisYearSum,label='2021')

ax1.legend()
ax1.grid()
ax1.set_ylabel('Antal tilfælde')
ax1.set_ylim(bottom=0)
ax1.set_xlim([np.datetime64('2021-01-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))

plt.tight_layout()
plt.savefig('figs/Compare2020and2021')


ax1.set_ylim(top=1500)
ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-07-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True,figsize=(14,20))
prevYearIndex = (testDates < np.datetime64('2021-01-01'))
prevYearDates = testDates[prevYearIndex]
prevYearSum = curdf[prevYearIndex]['NewPositive']
prevYearTest = curdf[prevYearIndex]['Tested']
# prevYearPos = curdf[prevYearIndex]['PosPct']
prevYearPos = 100 * np.divide(prevYearSum,prevYearTest)

thisYearIndex = (testDates >= np.datetime64('2021-01-01'))
thisYearDates = testDates[thisYearIndex]
thisYearSum = curdf[thisYearIndex]['NewPositive']
thisYearTest = curdf[thisYearIndex]['Tested']
# thisYearPos = curdf[thisYearIndex]['PosPct']
thisYearPos = 100 * np.divide(thisYearSum,thisYearTest)

meanWidth=7

ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')


ax2.plot(prevYearDates+np.timedelta64(365,'D'),prevYearTest,'b.:',linewidth=0.5,markersize=4)
ax2.plot(thisYearDates.values[:-1],thisYearTest.values[:-1],'r.:',linewidth=0.5,markersize=4)
ax2.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearTest,meanWidth),'b',label='2020')
ax2.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearTest.values[:-1],meanWidth),'r',label='2021')


ax3.plot(prevYearDates+np.timedelta64(365,'D'),prevYearPos,'b.:',linewidth=0.5,markersize=4)
ax3.plot(thisYearDates.values[:-1],thisYearPos.values[:-1],'r.:',linewidth=0.5,markersize=4)
ax3.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearPos,meanWidth),'b',label='2020')
ax3.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearPos.values[:-1],meanWidth),'r',label='2021')


xTicks = np.arange(np.datetime64('2021-01'),np.datetime64('2022-01'))
ax1.set_xticks(xTicks)
# plt.xticks(rotation=90)

ax1.legend()
ax1.grid()
ax2.grid()
ax3.grid()
ax1.set_ylabel('Antal tilfælde')
ax2.set_ylabel('Antal test')
ax3.set_ylabel('Positiv procent [%]')
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax3.set_ylim(bottom=0)
ax1.set_xlim([np.datetime64('2021-01-01'),np.datetime64('2022-01-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.tight_layout()
plt.savefig('figs/Compare2020and2021_All')

xTicks = np.arange(np.datetime64('2021-01-04'),np.datetime64('2022-01-08'),np.timedelta64(14,'D'))
ax1.set_xticks(xTicks)

ax3.set_ylim(top=1.5)

ax1.set_ylim(top=1400)
ax1.set_xlim([np.datetime64('2021-04-01'),np.datetime64('2021-08-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_All_Zoom')

xTicks = np.arange(np.datetime64('2021-01-04'),np.datetime64('2022-01-08'),np.timedelta64(14,'D'))
ax1.set_xticks(xTicks)

ax3.set_ylim(top=1)

ax1.set_ylim(top=1200)
ax1.set_xlim([np.datetime64('2021-06-01'),np.datetime64('2021-08-01')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()
plt.savefig('figs/Compare2020and2021_All_Zoom_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# import datetime
# d = "2021-W22"
# r = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w")
# print(np.datetime64(r))

In [17]:
fig,ax1 = plt.subplots(1,1)
meanWidth=7

allCase = curdf['NewPositive']
meanDates = rnTime(testDates,meanWidth)
allCaseMean = rnMean(allCase,meanWidth)

# ax1.plot(testDates,np.log(allCase),'k.:',linewidth=0.5,markersize=3)
# ax1.plot(rnTime(testDates,meanWidth),rnMean(np.log(allCase),meanWidth),'k')



# numPred = 30
# curT = np.arange(0,numPred)
# predDays = meanDates[-numPred:]
# curI0 = allCaseMean[-numPred]
iniPred = 60
endPred = iniPred - 14
curT = np.arange(0,iniPred-endPred)
predDays = meanDates[-iniPred:-endPred]
curI0 = allCaseMean[-iniPred]
# predDays = testDates[-numPred:]
# curI0 = allCase.values[-numPred]



# allRts = np.arange(9,12,0.25)/10
allRts = np.arange(10,13,0.5)/10

# curRt = 1
for curRt in allRts:
    curModel = ( (curRt - 1) * curT / 4.7 ) + np.log(curI0)
    ax1.plot(predDays,curModel,'--',label=str(curRt))



ax1.plot(testDates,np.log(allCase),'k.:',linewidth=0.5,markersize=3)
ax1.plot(rnTime(testDates,meanWidth),rnMean(np.log(allCase),meanWidth),'k')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))

# ax1.set_xlim(left=np.datetime64('2020-12-01'))
# ax1.set_ylim(bottom=5)


ax1.set_xlim(left=np.datetime64('2021-04-01'))
ax1.set_ylim(bottom=4,top=7.5)

ax1.legend()
# ax1.plot(prevYearDates+np.timedelta64(365,'D'),prevYearSum,'b.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(prevYearDates+np.timedelta64(365,'D'),meanWidth),rnMean(prevYearSum,meanWidth),'b',label='2020')
# ax1.plot(thisYearDates.values[:-1],thisYearSum.values[:-1],'r.:',linewidth=0.5,markersize=4)
# ax1.plot(rnTime(thisYearDates.values[:-1],meanWidth),rnMean(thisYearSum.values[:-1],meanWidth),'r',label='2021')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\ProgramData\Anaconda3\envs\main\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\Anaconda3\envs\main\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
